# Using STAR to align reads to the genome

Today we will be doing this using Jupyter Lab on jupyter.org/try using the yeast genome and some sample yeast fastq files. 

However the equivalent reference data and commands will be written for performing alignment with human reference genome.

To install STAR, we will use conda:
```
$ conda install -c bioconda STAR
```

Check that STAR was properly installed by simply typing `STAR` into your terminal:
```
$ STAR
```

You should see something like this:

```
Usage: STAR  [options]... --genomeDir /path/to/genome/index/   --readFilesIn R1.fq R2.fq
Spliced Transcripts Alignment to a Reference (c) Alexander Dobin, 2009-2019

For more details see:
<https://github.com/alexdobin/STAR>
<https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf>

To list all parameters, run STAR --help
```
    

## STAR documentation

Open the STAR user [manual](https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf). We will go through this briefly together to get an understanding of how to read documentation. 

## Build STAR reference genome index

Open UCSC genome [browser](https://genome.ucsc.edu/). The link to the specific annotations we will use is provided below, but first take a look through the website to see all the available annotations and features. We will go through this together.

### Download reference genome fasta files
Note: here we using yeast reference genome 

We will use UCSC to download the chromosome fasta files that are needed to build the STAR index. Use the same wget command followed by a copy of the web link address. The annotations are located [here](http://hgdownload.soe.ucsc.edu/goldenPath/sacCer3/bigZips/). Scroll to the bottom of the page and get the link for chromFa.tar.gz. We are going to first make a folder for the reference data so that we are organized. Once you have made the folder, move into it so your annotations will land in the proper place.

    $ mkdir -p ~/cmm262/references/yeast
    $ cd ~/cmm262/references/yeast
    $ wget http://hgdownload.soe.ucsc.edu/goldenPath/sacCer3/bigZips/chromFa.tar.gz

This will download a zipped file. You can then unzip the file with:

    $ tar -xvf chromFa.tar.gz
    
#### For human reference genome the equivalent is: 
```
$ wget http://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/chromFa.tar.gz
```

### Merge individual fasta file into one big one

Your folder should now contain many fasta files for each chromosome. You can merge them all together using cat (concatenate) and assigning the output to a new file called `yeast.fa` using `>`. This is the chromosome fasta file that you will need to use to generate the genome index.

    $ cat *.fa > yeast.fa
    
#### NOTE:
Safer practice to use different extension to avoid any unintended looping behavior
```
$ cat *.fa > yeast.fasta
```

Credit @Charles-Alexandre Roy

#### NOTE:
The `>` character saves the result of your command to a new file. In this case, we want to save the result of concatonating together all of the individual chromosome files into a giant one called yeast.fa

#### Q: 
Why did we have to indicate \*.fa instead of \* with the cat command?

### Download gene annotation file (.gtf)

In addition to chromosomal sequence information that we got from out fasta files we will also need gene annotations to make our index. We will use a pre-downloaded one from class GitHub. Use wget to obtain file:

``` 
$ wget  https://github.com/biom262/cmm262-2020/raw/master/Module_2/Data/yeast.gtf
```

#### For human reference genome: 

You can use gencode release (19) for genome build GRCh37 (hg19). We want the gtf file of the comprehensive gencode annotation for chromosomes.
Download the gtf annotation from gencode  [here](https://www.gencodegenes.org/human/release_29lift37.html)

Use the wget command again:

```
$ wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_29/GRCh37_mapping/gencode.v29lift37.annotation.gtf.gz
```

Unzip the file with gunzip (REMEMBER TO USE TABS TO AVOID TYPOS!):
```
$ gunzip gencode.v29lift37.annotation.gtf.gz
```
AKA
```
$ gunzip genc<tab>
```
*Note - the unzip is different than above, because the above file was tar.gz which required tar -xvf to unzip. This one is only .gz, so it can be unzipped with gunzip.*

### Run STAR index command

Refer to the STAR manual for a description of this step. What flags do you need to include? Work with your partner to decide what will be important given the information in the manual.

Once you have decided on what your STAR command should look like go back through and PAY CLOSE ATTENTION TO FULL PATHS OF FILES. You have downloaded the necessary annotations already, make sure the paths to those files are correct in your command. I recommend using tab-complete to make the full path and then copying and pasting them directly into your script. Remember you can display the path with pwd.

What did you learn about the --genomeDir flag from the documentation? It looks like you need to make a folder where the output can go before we run the script. Let's make that now and make sure we have the path correct in our script before running.

```
$ mkdir ~/cmm262/references/yeast/star
```


As an example, my command looks like this:
```
$ STAR --runThreadN 1 --runMode genomeGenerate --genomeDir ~/cmm262/references/yeast/star --genomeFastaFiles ~/cmm262/references/yeast/yeast.fa --sjdbGTFFile ~/cmm262/references/yeast/yeast.gtf
```

#### Dealing with Errors

You submitted your command, let's look at the example error that was output:

    EXITING: FATAL INPUT ERROR: unrecoginzed parameter name "sjdbGTFFile" in input "Command-Line-Initial"
    SOLUTION: use correct parameter name (check the manual)

    Jul 21 14:19:02 ...... FATAL ERROR, exiting
    
    
Solution... Go back and check that argument with the GTF filename, it looks like there was a typo, the second F should not be capitalized

```
$ STAR --runThreadN 1 --runMode genomeGenerate --genomeDir ~/cmm262/references/yeast/star --genomeFastaFiles ~/cmm262/references/yeast/yeast.fa --sjdbGTFfile ~/cmm262/references/yeast/yeast.gtf
```

## Mapping reads to genome

Once your job is complete you can move onto the next step of mapping your reads to the genome. Information on this step can be found under "Running mapping jobs" in the basic options: [manual](https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf)

Let's get some sample fastq files to do the alignment with from the GitHub. First make a separate directory for the raw data:
```
$ mkdir ~/cmm262/raw_data
```

Then download 2 fastq files for one sample (Read 1 and Read 2)
```
$ cd ~/cmm262/raw_data
$ wget  https://github.com/biom262/cmm262-2020/raw/master/Module_2/Data/yeast_R1.fastq
$ wget  https://github.com/biom262/cmm262-2020/raw/master/Module_2/Data/yeast_R2.fastq
$ cd ..
```

Now let's make a directory for the output:
```
$ mkdir ~/cmm262/star_alignment
```

Look over the STAR manual to decide on how to set up this step.

As an example, here is the command I will be using:
```
$ STAR --runThreadN 1 --genomeDir ~/cmm262/references/yeast/star  --readFilesIn ~/cmm262/raw_data/yeast_R1.fastq ~/scratch/raw_data/yeast_R2.fastq --outFilterType BySJout --outFilterMultimapNmax 10 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --outFilterMismatchNmax 4 --alignIntronMin 20 --alignIntronMax 5000 --alignMatesGapMax 5000 --outFileNamePrefix ~/cmm262/star_alignment/yeast_example_
``` 

#### Q:
What would you do if you had fastq.gz files instead of fastq files?

### Take a look at the log file

In section 4 Output Files of the STAR manual, take a look at the different output files to expect and view each one with less to see how your run went. 

Remember you specified the path of where these files would end up with your STAR submission script above.

#### Q:
Did the run complete?

### Q:
What is the difference between a sam and a bam file?*

We can use the example in GitHub to see what a typical final log output file should look like: https://github.com/biom262/cmm262-2020/blob/module_2_dn/Module_2/Data/yeast_example_Log.final.out